In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/fake-and-real-news-dataset/True.csv
/kaggle/input/fake-and-real-news-dataset/Fake.csv


In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
import re
import string

In [3]:
true_df = pd.read_csv('/kaggle/input/fake-and-real-news-dataset/True.csv')
true_df.head()

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [4]:
fake_df = pd.read_csv('/kaggle/input/fake-and-real-news-dataset/Fake.csv')
fake_df.head()

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [5]:
print("true shape : ",true_df.shape)
print("fake shape : ",fake_df.shape)

true shape :  (21417, 4)
fake shape :  (23481, 4)


#### Add the column **label**   as target feature

In [6]:
true_df['label'] = 1
fake_df['label'] = 0

In [7]:
print("true shape : ",true_df.shape)
print("fake shape : ",fake_df.shape)

true shape :  (21417, 5)
fake shape :  (23481, 5)


In [8]:
# Removing last 10 rows for manual testing
df_fake_manual_testing = fake_df.tail(10)
for i in range(23480,23470,-1):
    fake_df.drop([i], axis = 0, inplace = True)
    
    
df_true_manual_testing = true_df.tail(10)
for i in range(21416,21406,-1):
    true_df.drop([i], axis = 0, inplace = True)

In [9]:
import warnings
warnings.filterwarnings("ignore")

In [10]:
df_fake_manual_testing["label"] = 0
df_true_manual_testing["label"] = 1

In [11]:
df_fake_manual_testing.head(10)


,title,text,subject,date,label
23471,Seven Iranians freed in the prisoner swap have...,"21st Century Wire says This week, the historic...",Middle-east,"January 20, 2016",0
23472,#Hashtag Hell & The Fake Left,By Dady Chery and Gilbert MercierAll writers ...,Middle-east,"January 19, 2016",0
23473,Astroturfing: Journalist Reveals Brainwashing ...,Vic Bishop Waking TimesOur reality is carefull...,Middle-east,"January 19, 2016",0
23474,The New American Century: An Era of Fraud,Paul Craig RobertsIn the last years of the 20t...,Middle-east,"January 19, 2016",0
23475,Hillary Clinton: ‘Israel First’ (and no peace ...,Robert Fantina CounterpunchAlthough the United...,Middle-east,"January 18, 2016",0
23476,McPain: John McCain Furious That Iran Treated ...,21st Century Wire says As 21WIRE reported earl...,Middle-east,"January 16, 2016",0
23477,JUSTICE? Yahoo Settles E-mail Privacy Class-ac...,21st Century Wire says It s a familiar theme. ...,Middle-east,"January 16, 2016",0
23478,Sunnistan: US and Allied ‘Safe Zone’ Plan to T...,Patrick Henningsen 21st Century WireRemember ...,Middle-east,"January 15, 2016",0
23479,How to Blow $700 Million: Al Jazeera America F...,21st Century Wire says Al Jazeera America will...,Middle-east,"January 14, 2016",0
23480,10 U.S. Navy Sailors Held by Iranian Military ...,21st Century Wire says As 21WIRE predicted in ...,Middle-east,"January 12, 2016",0


In [12]:
df_true_manual_testing.head(10)


,title,text,subject,date,label
21407,"Mata Pires, owner of embattled Brazil builder ...","SAO PAULO (Reuters) - Cesar Mata Pires, the ow...",worldnews,"August 22, 2017",1
21408,"U.S., North Korea clash at U.N. forum over nuc...",GENEVA (Reuters) - North Korea and the United ...,worldnews,"August 22, 2017",1
21409,"U.S., North Korea clash at U.N. arms forum on ...",GENEVA (Reuters) - North Korea and the United ...,worldnews,"August 22, 2017",1
21410,Headless torso could belong to submarine journ...,COPENHAGEN (Reuters) - Danish police said on T...,worldnews,"August 22, 2017",1
21411,North Korea shipments to Syria chemical arms a...,UNITED NATIONS (Reuters) - Two North Korean sh...,worldnews,"August 21, 2017",1
21412,'Fully committed' NATO backs new U.S. approach...,BRUSSELS (Reuters) - NATO allies on Tuesday we...,worldnews,"August 22, 2017",1
21413,LexisNexis withdrew two products from Chinese ...,"LONDON (Reuters) - LexisNexis, a provider of l...",worldnews,"August 22, 2017",1
21414,Minsk cultural hub becomes haven from authorities,MINSK (Reuters) - In the shadow of disused Sov...,worldnews,"August 22, 2017",1
21415,Vatican upbeat on possibility of Pope Francis ...,MOSCOW (Reuters) - Vatican Secretary of State ...,worldnews,"August 22, 2017",1
21416,Indonesia to buy $1.14 billion worth of Russia...,JAKARTA (Reuters) - Indonesia will buy 11 Sukh...,worldnews,"August 22, 2017",1


In [13]:
df_manual_testing = pd.concat([df_fake_manual_testing,df_true_manual_testing], axis = 0)
df_manual_testing.to_csv("manual_testing.csv")

### Concatinating the true and fake dataframes

In [14]:
df_merge = pd.concat([true_df, fake_df], axis = 0)
df_merge.head()

,title,text,subject,date,label
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017",1
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017",1
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017",1
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017",1
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017",1


In [15]:
df_merge.columns

Index(['title', 'text', 'subject', 'date', 'label'], dtype='object')

## Remove the uneccessay columns

In [16]:
data = df_merge.drop(['title',  'subject', 'date'],axis = 1)

In [17]:
data.head()

,text,label
0,WASHINGTON (Reuters) - The head of a conservat...,1
1,WASHINGTON (Reuters) - Transgender people will...,1
2,WASHINGTON (Reuters) - The special counsel inv...,1
3,WASHINGTON (Reuters) - Trump campaign adviser ...,1
4,SEATTLE/WASHINGTON (Reuters) - President Donal...,1


In [18]:
data.isnull().sum()

text     0
label    0
dtype: int64

## Random shuffling the dataframes

In [19]:
data = data.sample(frac =1)
data.head()

,text,label
2775,Kellyanne Conway tried to baffle George Stepha...,0
14495,BAGHDAD (Reuters) - Iraqi forces on Friday cap...,1
2673,WASHINGTON (Reuters) - The top Democrat on the...,1
1090,A horrifying incident took place in the early ...,0
15903,The story below from the liberal New York Time...,0


In [20]:
data.reset_index(inplace = True)
data.drop(['index'], axis = 1, inplace = True)

In [21]:
data.head()

,text,label
0,Kellyanne Conway tried to baffle George Stepha...,0
1,BAGHDAD (Reuters) - Iraqi forces on Friday cap...,1
2,WASHINGTON (Reuters) - The top Democrat on the...,1
3,A horrifying incident took place in the early ...,0
4,The story below from the liberal New York Time...,0


## Function to preprocess the text

In [22]:
import re
import string
from nltk.corpus import stopwords

def preprocess(text):
    # Convert text to lowercase
    text = text.lower()
    
    # Remove square brackets and their contents
    text = re.sub('\[.*?\]', '', text)
    
    # Remove non-word characters
    text = re.sub("\\W", " ", text)
    
    # Remove URLs
    text = re.sub('https?://\S+|www\.\S+', '', text)
    
    # Remove HTML tags
    text = re.sub('<.*?>', '', text)
    
    # Remove punctuation
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    
    # Remove newline characters
    text = re.sub('\n', '', text)
    
    # Remove alphanumeric characters
    text = re.sub('\w*\d\w*', '', text)    
    
    # Remove stop words
    stop_words = set(stopwords.words('english'))
    tokens = text.split()
    filtered_tokens = [token for token in tokens if token not in stop_words]
    text = ' '.join(filtered_tokens)
    
    return text


In [23]:
data['cleaned_text'] = data['text'].apply(preprocess)

In [24]:
data.head()

,text,label,cleaned_text
0,Kellyanne Conway tried to baffle George Stepha...,0,kellyanne conway tried baffle george stephanop...
1,BAGHDAD (Reuters) - Iraqi forces on Friday cap...,1,baghdad reuters iraqi forces friday captured b...
2,WASHINGTON (Reuters) - The top Democrat on the...,1,washington reuters top democrat u house repres...
3,A horrifying incident took place in the early ...,0,horrifying incident took place early hours mon...
4,The story below from the liberal New York Time...,0,story liberal new york times hysterical rex ti...


In [25]:
data.drop(['text'], axis = 1, inplace = True)

In [26]:
data.head()

,label,cleaned_text
0,0,kellyanne conway tried baffle george stephanop...
1,1,baghdad reuters iraqi forces friday captured b...
2,1,washington reuters top democrat u house repres...
3,0,horrifying incident took place early hours mon...
4,0,story liberal new york times hysterical rex ti...


In [27]:
# Extract the 'label' column
label_column = data.pop('label')

# Re-insert the 'label' column at the end
data['label'] = label_column
data.head()

,cleaned_text,label
0,kellyanne conway tried baffle george stephanop...,0
1,baghdad reuters iraqi forces friday captured b...,1
2,washington reuters top democrat u house repres...,1
3,horrifying incident took place early hours mon...,0
4,story liberal new york times hysterical rex ti...,0


## Defining dependent and independent variables

In [28]:
X = data['cleaned_text']
y = data['label']

## Splitting Training and Testing

In [29]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

## Convert text to vectors

In [30]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorization = TfidfVectorizer()
xv_train = vectorization.fit_transform(X_train)
xv_test = vectorization.transform(X_test)

## LOGISTIC REGRESSION

In [31]:
from sklearn.linear_model import LogisticRegression

LR = LogisticRegression()
LR.fit(xv_train, y_train)

LogisticRegression()

In [32]:
pred_lr = LR.predict(xv_test)
LR.score(xv_test,y_test)

0.9882352941176471

In [33]:
print(classification_report(y_test, pred_lr))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99      5786
           1       0.99      0.99      0.99      5434

    accuracy                           0.99     11220
   macro avg       0.99      0.99      0.99     11220
weighted avg       0.99      0.99      0.99     11220



## Decision Tree Classification


In [34]:
from sklearn.tree import DecisionTreeClassifier


DT = DecisionTreeClassifier()
DT.fit(xv_train,y_train)


DecisionTreeClassifier()

In [35]:
pred_dt = DT.predict(xv_test)
DT.score(xv_test, y_test)

0.9950089126559715

In [36]:
print(classification_report(y_test, pred_dt))

              precision    recall  f1-score   support

           0       0.99      1.00      1.00      5786
           1       1.00      0.99      0.99      5434

    accuracy                           1.00     11220
   macro avg       1.00      0.99      1.00     11220
weighted avg       1.00      1.00      1.00     11220



## Gradient Boosting Classifier

In [37]:
from sklearn.ensemble import GradientBoostingClassifier

GBC = GradientBoostingClassifier(random_state=0)
GBC.fit(xv_train, y_train)

GradientBoostingClassifier(random_state=0)

In [38]:
pred_gbc = GBC.predict(xv_test)
GBC.score(xv_test, y_test)

0.9959893048128342

In [39]:
print(classification_report(y_test, pred_gbc))

              precision    recall  f1-score   support

           0       1.00      0.99      1.00      5786
           1       0.99      1.00      1.00      5434

    accuracy                           1.00     11220
   macro avg       1.00      1.00      1.00     11220
weighted avg       1.00      1.00      1.00     11220



## Random Forest Classifier


In [40]:
from sklearn.ensemble import RandomForestClassifier

RFC = RandomForestClassifier(random_state=0)
RFC.fit(xv_train, y_train)

RandomForestClassifier(random_state=0)

In [41]:
pred_rfc = RFC.predict(xv_test)
RFC.score(xv_test, y_test)

0.9934046345811052

In [42]:
print(classification_report(y_test, pred_rfc))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99      5786
           1       0.99      0.99      0.99      5434

    accuracy                           0.99     11220
   macro avg       0.99      0.99      0.99     11220
weighted avg       0.99      0.99      0.99     11220



In [43]:
def output_lable(n):
    if n == 0:
        return "Fake News"
    elif n == 1:
        return "Not A Fake News"

In [44]:
def manual_testing(news):
    testing_news = {"text":[news]}
    new_def_test = pd.DataFrame(testing_news)
    new_def_test["text"] = new_def_test["text"].apply(preprocess) 
    new_x_test = new_def_test["text"]
    new_xv_test = vectorization.transform(new_x_test)
    pred_LR = LR.predict(new_xv_test)
    pred_DT = DT.predict(new_xv_test)
    pred_GBC = GBC.predict(new_xv_test)
    pred_RFC = RFC.predict(new_xv_test)

    return print("\n\nLR Prediction: {} \nDT Prediction: {} \nGBC Prediction: {} \nRFC Prediction: {}".format(output_lable(pred_LR[0]),                                                                                                       output_lable(pred_DT[0]), 
                                                                                                              output_lable(pred_GBC[0]), 
                                                                                                              output_lable(pred_RFC[0])))

In [45]:
# news = str(input())
news = '''MOSCOW (Reuters) - Russiaâ€™s former ambassador to Washington, Sergei Kislyak,
said on Saturday his conversations with former White House national security adviser Michael Flynn had been 
transparent and focused on matters of U.S.-Russia cooperation. Kislyak ended his tenure in Washington in July 
but remains a key figure in ongoing U.S. investigations into Moscowâ€™s alleged meddling in the 2016 presidential election.
Flynn was forced to resign in February after it became known that he had failed to disclose the content of 
conversations he had with Kislyak and misled U.S. Vice-President Mike Pence about their meetings. â€œWe only 
spoke about the most simple things ... but the communication was completely correct, calm, absolutely transparent.
In any case, there were no secrets on our side,â€ Kislyak said during a panel discussion on Russian television. â€œThere are a number of issues which are important for cooperation between Russia and the United States - most of all, terrorism. And that was one of the things we discussed
'''
manual_testing(news)



LR Prediction: Not A Fake News 
DT Prediction: Not A Fake News 
GBC Prediction: Not A Fake News 
RFC Prediction: Not A Fake News


In [46]:
# news = str(input())
news = '''Hey Facebook, As some of you may know, I'm Bill Gates. If you click that share link, I will give you $5,000. I always deliver, I mean, I brought you Windows XP, right?'''
manual_testing(news)



LR Prediction: Fake News 
DT Prediction: Fake News 
GBC Prediction: Fake News 
RFC Prediction: Fake News


# Thanks for coming here.
#### If you find it useful upvote me 